In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
movies_genres = pd.read_csv("genre_data.csv")

In [3]:
feature_id = pd.read_csv("feature_taxonomy.csv")

In [4]:
longform = pd.read_csv("feature_data_longform.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'feature_data_longform.csv'

In [ ]:
longform

In [5]:
movies_genres

,movie_id,movie_name,genre
0,tt11137336,The Beauty,Romance
1,tt1309557,Eamon,"Comedy, Romance"
2,tt0038504,Earl Carroll Sketchbook,"Comedy, Music, Romance"
3,tt3297804,Art Idol,"Comedy, Drama, Romance"
4,tt0140917,O Corpo Ardente,"Drama, Romance"
...,...,...,...
7995,tt1464605,Ying lie qian qiu,"Drama, War"
7996,tt0780131,Journey to Chazzabeh,War
7997,tt0352615,Nan zheng bei zhan,War
7998,tt0017623,Am Rande der Welt,"Drama, War"


In [ ]:
feature_id

In [ ]:
threes = longform[longform['trigger'] == 3] 

In [ ]:
cleantrig = threes.drop(columns=['imdb_id'])
cleantrig

In [ ]:
triggersum = pd.DataFrame(cleantrig.groupby('feature_id')['trigger'].sum().sort_values(ascending=True))

In [ ]:
triggersumOUT = pd.merge(triggersum, feature_id, on='feature_id')

In [ ]:
triggersumOUT[triggersumOUT['feature_id'] == 440]

In [ ]:
longform.head(30)

In [ ]:
triggersumOUT.tail(30)

In [ ]:
movies_genres

In [ ]:
movies_genres[4000:4020]

In [ ]:
!pip install requests
import requests
import json

def get_movie_rating(movie_title, api_key):
    """
    Fetches the vote average for a movie from TMDb by title.
    """
    # The API endpoint for searching movies
    url = "https://api.themoviedb.org/3/search/movie"

    # Parameters for the GET request
    params = {
        'api_key': api_key,
        'query': movie_title
    }

    try:
        # Send the GET request to the API
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the JSON response into a Python dictionary
        data = response.json()

        # Check if any results were found
        if data['results']:
            # Get the first movie from the results list
            first_result = data['results'][0]
            title = first_result['title']
            # The rating is stored in the 'vote_average' field
            rating = first_result['vote_average']
            print(f"The rating for '{title}' is: {rating} (out of 10)")
        else:
            print(f"No movie found with the title '{movie_title}'")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    except json.JSONDecodeError:
        print("Error decoding JSON response.")

In [ ]:
# 1. Setup API Key and TMDB Base URL
TMDB_API_KEY = "c4202f387cff7736aaa742055815bd72"
BASE_URL = "https://api.themoviedb.org/3"

def get_movie_certification(title, country="US"):
    # 2. Search for the movie to get its ID
    search_url = f"{BASE_URL}/search/movie?api_key={TMDB_API_KEY}&query={title}"
    search_response = requests.get(search_url).json()
    
    if not search_response['results']:
        return "Movie not found"
    
    movie_id = search_response['results'][0]['id']
    
    # 3. Get release dates/certifications for the movie
    release_url = f"{BASE_URL}/movie/{movie_id}/release_dates?api_key={TMDB_API_KEY}"
    release_response = requests.get(release_url).json()
    
    # 4. Find the specific country's rating
    for result in release_response['results']:
        if result['iso_3166_1'] == country:
            # Iterate through release types (3 is theatrical)
            for release in result['release_dates']:
                if release['certification']:
                    return release['certification']
                    
    return "No rating found"

In [ ]:
# Example Usage
movie_title = "Starsky & Hutch"
rating = get_movie_certification(movie_title)
print(f"The rating for '{movie_title}' in US is: {rating}")

In [ ]:
movies_genres['Rating'] = np.zeros(8000)
for i in range(len(movies_genres)): 
    try:
        movies_genres['Rating'][i] = get_movie_certification(movies_genres['movie_name'][i])
    except:
        movies_genres['Rating'][i] = -1